In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import scipy.ndimage.interpolation
from PIL import Image

In [4]:
mnist = input_data.read_data_sets("mnist", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
n_classes=10
batch_size=128
x=tf.placeholder('float',[None,784])
y=tf.placeholder('float')

In [6]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
def maxpool2d(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
def conv2d_transpose(x,W,batch_size,output,out):
    return tf.nn.conv2d_transpose(x,W,[batch_size,output,output,out],strides=[1,2,2,1],padding='SAME')

In [7]:
def discriminator_convolution_gan(x):
    weights={
        'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
        'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
        'W_fc':tf.Variable(tf.random_normal([7*7*64,1024])),
        'W_out':tf.Variable(tf.random_normal([1024,n_classes])),
        'W_prob':tf.Variable(tf.random_normal([n_classes,1]))
    }
    biases={
        'b_conv1':tf.Variable(tf.random_normal([32])),
        'b_conv2':tf.Variable(tf.random_normal([64])),
        'b_fc':tf.Variable(tf.random_normal([1024])),
        'b_out':tf.Variable(tf.random_normal([n_classes])),
        'b_prob':tf.Variable(tf.random_normal([1]))
    }
    x=tf.reshape(x,shape=[-1,28,28,1])
    conv1=conv2d(x,weights['W_conv1'])+biases['b_conv1']
    conv1 = tf.layers.batch_normalization(conv1)
    conv1=tf.nn.leaky_relu(conv1)
    conv1=maxpool2d(conv1)
    conv2=conv2d(conv1,weights['W_conv2'])+biases['b_conv2']
    conv2 =tf.layers.batch_normalization(conv2)
    conv2=tf.nn.leaky_relu(conv2)
    conv2=maxpool2d(conv2)
    fc=tf.reshape(conv2,[-1,7*7*64])
    fc=tf.nn.leaky_relu(tf.matmul(fc,weights['W_fc'])+biases['b_fc'])
    output=tf.nn.leaky_relu(tf.matmul(fc,weights['W_out'])+biases['b_out'])
    prob=tf.nn.sigmoid(tf.matmul(output,weights['W_prob'])+biases['b_prob'])
    return prob


In [8]:

#z=np.random.uniform(-1,1,size=[128,100])
def generator_convolution_gan(z,batch_size):
    weights={
        'W_in':tf.Variable(tf.random_normal([100,1024])),
        'W_fc':tf.Variable(tf.random_normal([1024,3136])),
        'W_conv1':tf.Variable(tf.random_normal([5,5,32,64])),
        'W_conv2':tf.Variable(tf.random_normal([5,5,1,32]))
    }
    biases={
        'b_in':tf.Variable(tf.random_normal([1024])),
        'b_fc':tf.Variable(tf.random_normal([3136])),
        'b_conv1':tf.Variable(tf.random_normal([32])),
        'b_conv2':tf.Variable(tf.random_normal([1]))
        
    }
    z=tf.reshape(z,shape=[-1,100])
    input_g=tf.nn.relu(tf.matmul(z,weights['W_in'])+biases['b_in'])
    fc=tf.nn.relu(tf.matmul(input_g,weights['W_fc'])+biases['b_fc'])
    fc=tf.reshape(fc,[-1,7,7,64])
    conv1=conv2d_transpose(fc,weights['W_conv1'],batch_size,14,32)+biases['b_conv1']
    conv1 = tf.layers.batch_normalization(conv1)
    conv1=tf.nn.relu(conv1)
    #conv1=maxpool2d(conv1)
    #conv1=tf.reshape(fc,[-1,14,14,32])
    conv2=conv2d_transpose(conv1,weights['W_conv2'],batch_size,28,1)+biases['b_conv2']
    conv2 = tf.layers.batch_normalization(conv2)
    conv2=tf.nn.tanh(conv2)
    #conv2=maxpool2d(conv2)
    conv2=tf.reshape(conv2,[-1,784])
    return conv2

In [37]:
y_true=tf.placeholder('float')
y_fake=tf.placeholder('float')
#samples=tf.placeholder('float')

z=tf.placeholder('float',[None,100])
def train_neural_network(x):
    D_real=discriminator_convolution_gan(x)
    G=generator_convolution_gan(z,batch_size)
    D_fake=discriminator_convolution_gan(G)
    D_real_cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real,labels=y_true))
    D_fake_cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake,labels=y_fake))
    G_loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake,labels=y_true))
    D_loss=D_real_cost+D_fake_cost
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        optimizer_D=tf.train.AdamOptimizer().minimize(D_loss)#Need to change the optimiser
        optimizer_G=tf.train.AdamOptimizer().minimize(G_loss)#Need to change the optimiser
    
    hm_epochs=3
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        samples=np.zeros([128,28,28])
        for epoch in range(hm_epochs):
            epoch_loss=0
            epoch_loss_g=0
            for i in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y=mnist.train.next_batch(batch_size)
                epoch_z=np.random.uniform(-1,1,size=[batch_size,100])
                _,c=sess.run([optimizer_D,D_loss],feed_dict={x:epoch_x,y_true:np.ones([batch_size,1]),y_fake:np.zeros([batch_size,1]),z:epoch_z})
                _,g,samples_new=sess.run([optimizer_G,G_loss,G],feed_dict={z:epoch_z,y_true:np.ones([batch_size,1])})
                samples_new=np.array(samples_new.reshape(batch_size,28,28))
                samples=np.append(samples,samples_new,axis=0)
                
                epoch_loss=epoch_loss+c
                epoch_loss_g=epoch_loss_g+g
            print('Epoch',epoch,'completed out of',hm_epochs,'loss:',epoch_loss)
            print('Epoch',epoch,'completed out of',hm_epochs,'loss_g:',epoch_loss_g)
        return samples
            #print(sess.run(discriminator.hidden_1_layer['weights'])) This command gives the weights
samples=train_neural_network(x)



#print(samples.shape)

Epoch 0 completed out of 3 loss: 708.9436501264572
Epoch 0 completed out of 3 loss_g: 134.38926404714584
Epoch 1 completed out of 3 loss: 697.7785792350769
Epoch 1 completed out of 3 loss_g: 134.38926404714584
Epoch 2 completed out of 3 loss: 697.7785792350769
Epoch 2 completed out of 3 loss_g: 134.38926404714584


In [38]:
samples_test=np.array(samples)

In [87]:
z=tf.placeholder('float',[None,100])
#z=np.random.uniform(-1,1,size=[128,100])
def generator_convolution_gan(z):
    weights={
        'W_in':tf.Variable(tf.random_normal([100,1024])),
        'W_fc':tf.Variable(tf.random_normal([1024,7*7*64])),
        'W_conv1':tf.Variable(tf.random_normal([5,5,64,32])),
        'W_conv2':tf.Variable(tf.random_normal([5,5,32,1]))
    }
    biases={
        'b_in':tf.Variable(tf.random_normal([1024])),
        'b_fc':tf.Variable(tf.random_normal([64])),
        'b_conv1':tf.Variable(tf.random_normal([32])),
        'b_conv2':tf.Variable(tf.random_normal([1]))
        
    }
    input_g=tf.nn.relu(tf.matmul(z,weights['W_in'])+biases['b_in'])
    fc=tf.nn.relu(tf.matmul(input_g,weights['W_fc'])+biases['b_fc'])
    fc=tf.reshape(fc,[-1,7,7,64])
    conv1=conv2d(fc,weights['W_conv1'])+biases['b_conv1']
    conv1 = tf.layers.batch_normalization(conv1)
    conv1=tf.nn.relu(conv1)
    conv1=maxpool2d(conv1)
    
    conv2=conv2d(conv1,weights['W_conv2'])+biases['b_conv2']
    conv2 = tf.layers.batch_normalization(conv2)
    conv2=tf.nn.relu(conv2)
    conv2=maxpool2d(conv2)
    return conv1


In [4]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 100

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    # OLD VERSION:
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    # NEW:
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        # OLD:
        #sess.run(tf.initialize_all_variables())
        # NEW:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

train_neural_network(x)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 0 completed out of 10 loss: 1820493.5403060913
Epoch 1 completed out of 10 loss: 416692.27976226807
Epoch 2 completed out of 10 loss: 223705.87715387344
Epoch 3 completed out of 10 loss: 131737.47967386246
Epoch 4 completed out of 10 loss: 81825.28821472079
Epoch 5 completed out of 10 loss: 50157.09018534573
Epoch 6 completed out of 10 loss: 35186.85095919743
Epoch 7 completed out of 10 loss: 25908.77319342326
Epoch 8 completed out of 10 loss: 23067.286876805127
Epoch 9 completed out of 10 loss: 19883.85368725538
Accuracy: 0.9498


In [100]:
z=tf.placeholder('float',[None,100])
gg=generator_convolution_gan(z)
ggd=discriminator_convolution_gan(gg)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    epoch_zz=np.random.uniform(-1,1,size=[128,100])
    ggd=sess.run([ggd],feed_dict={z:epoch_zz})
ggd=np.array(ggd)
print(ggd.shape)
print(ggd)

(1, 128, 1)
[[[0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [1.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [1.]
  [0.]
  [0.]
  [1.]
  [0.]
  [0.]
  [0.]
  [0.]
  [1.]
  [1.]
  [0.]
  [0.]
  [0.]
  [1.]]]


In [26]:
a=np.array([])

In [21]:
a


array([], dtype=float64)

In [27]:
b=np.ones([3,4])

In [33]:
a=np.vstack((a,b))

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [29]:
a

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [35]:
a[3]

1.0

In [36]:
a[3,4]


IndexError: too many indices for array

In [39]:
a[3:4]

array([1.])

In [40]:
a[3:9]

array([1., 1., 1., 1., 1., 1.])

In [53]:
a=np.zeros([3,4])

In [54]:
a

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [56]:
a[1:,1:]

array([[0., 0., 0.],
       [0., 0., 0.]])

In [39]:
samples.shape

(164864, 28, 28)

In [64]:
samples_test

array([ 0.,  0.,  0., ...,  1.,  1., -1.])

In [42]:
samples_test.shape

(164864, 28, 28)

In [12]:
samples_test=(0.5*samples_test)+0.5

In [18]:
samples_test[128:,0:,0:]=255*((0.5*samples_test[128:,0:,0:])+0.5)

In [19]:
print(samples_test[128:129,0:,0:])

[[[255.   0.   0.   0.   0.   0. 255. 255. 255. 255. 255.   0. 255. 255.
   255.   0. 255.   0. 255.   0. 255. 255.   0.   0.   0. 255. 255.   0.]
  [  0.   0.   0. 255.   0. 255.   0. 255. 255. 255.   0. 255.   0. 255.
     0. 255. 255.   0.   0. 255.   0. 255.   0. 255.   0. 255.   0. 255.]
  [255.   0. 255. 255.   0.   0.   0. 255. 255.   0.   0.   0.   0. 255.
     0. 255.   0.   0.   0.   0.   0. 255.   0.   0.   0. 255. 255.   0.]
  [  0.   0.   0. 255. 255. 255.   0.   0.   0. 255.   0. 255. 255. 255.
   255. 255. 255. 255.   0. 255. 255. 255. 255.   0. 255. 255. 255.   0.]
  [  0. 255. 255. 255.   0. 255.   0. 255. 255. 255. 255. 255. 255. 255.
     0. 255. 255. 255. 255.   0.   0. 255. 255. 255. 255.   0.   0.   0.]
  [  0. 255.   0. 255. 255. 255. 255. 255.   0. 255.   0.   0.   0.   0.
     0.   0.   0.   0.   0. 255.   0.   0.   0. 255. 255. 255.   0. 255.]
  [255.   0.   0. 255. 255. 255.   0.   0. 255. 255.   0. 255.   0. 255.
     0.   0.   0.   0. 255. 255. 255. 255. 25

In [43]:
im=samples_test[128:,0:,0:].astype(np.uint8)

In [55]:
for i in range(im[0:,0,0].shape[0]):
    img=im[i,0:,0:]
    if i%(859*16)==0:
        img=img.reshape([28,28])
        img=Image.fromarray(img)
        img = img.resize((400, 400))
        img.save('testing{}_{}.png'.format(i/(859*16),np.round(i/(859*16*8))))


In [27]:
im.shape

(28, 28)

In [28]:
im = Image.fromarray(im)

In [29]:
im.show()

In [50]:
im[0:,0,0].shape[0]

164736